*Data Model Mapper Admin App*



Run the cleanup code below if needed. - Don't forget to update <your warehouse name> with a warehouse name

In [ ]:
/*Cleanup
use role accountadmin;
use warehouse app_wh;
drop database if exists data_model_mapper_admin_app;
--Only run if you are using file pickler notebook
drop database if exists data_model_mapper_test_db
*/

Now let's create the application package and necessary tables

In [ ]:
/* create database*/
create database data_model_mapper_admin_app
comment='{"origin":"sf_sit","name":"dmm","version":{"major":2, "minor":0},"attributes":{"component":"dmm_admin"}}';

/* admin holds all base tables, managed by administrator */
create or replace schema data_model_mapper_admin_app.admin
comment='{"origin":"sf_sit","name":"dmm","version":{"major":2, "minor":0},"attributes":{"component":"dmm_admin"}}';
drop schema if exists data_model_mapper_admin_app.public;


/* target collection - group of entities that define a data model, can be analogous to "product" */
create or replace table data_model_mapper_admin_app.admin.target_collection (
    target_collection_name varchar,
    version varchar,
    last_updated_timestamp timestamp default current_timestamp(),
    constraint pkey primary key (target_collection_name, version) not enforced
) comment='{"origin":"sf_sit","name":"dmm","version":{"major":2, "minor":0},"attributes":{"component":"dmm_admin"}}';


/* target entity - one target collection to many entities, can be analogous to "table" */
create or replace table data_model_mapper_admin_app.admin.target_entity (
    target_collection_name varchar,
    version varchar,
    target_entity_name varchar,
    last_updated_timestamp timestamp default current_timestamp(),
    constraint pkey primary key (target_collection_name, version, target_entity_name) not enforced,
    constraint fkey_1 foreign key (target_collection_name, version)
        references data_model_mapper_admin_app.admin.target_collection (target_collection_name, version) not enforced
) comment='{"origin":"sf_sit","name":"dmm","version":{"major":2, "minor":0},"attributes":{"component":"dmm_admin"}}';


/* target entity attribute - one entity to many attributes, can be analogous to "column" */
create or replace table data_model_mapper_admin_app.admin.target_entity_attribute (
    target_collection_name varchar,
    version varchar,
    target_entity_name varchar,
    target_entity_attribute_name varchar,
    target_attribute_properties object,
    include_in_entity boolean default true,
    last_updated_timestamp timestamp default current_timestamp(),
    constraint pkey primary key (target_collection_name, version, target_entity_name, target_entity_attribute_name) not enforced,
    constraint fkey_1 foreign key (target_collection_name, version, target_entity_name)
        references data_model_mapper_admin_app.admin.target_entity (target_collection_name, version, target_entity_name) not enforced
) comment='{"origin":"sf_sit","name":"dmm","version":{"major":2, "minor":0},"attributes":{"component":"dmm_admin"}}';


Add our procedure to get the attributes for entities

In [ ]:
create or replace procedure data_model_mapper_admin_app.admin.generate_attributes(attr_table varchar, target_collection_name varchar, target_entity_name varchar, version varchar)
returns string
language python
runtime_version=3.11
packages=('snowflake-snowpark-python')
handler='generate_attributes'
comment='{"origin":"sf_sit","name":"dmm","version":{"major":2, "minor":0},"attributes":{"component":"dmm_admin"}}'
as
$$
import snowflake.snowpark
from snowflake.snowpark.functions import call_udf, col, current_timestamp, when_matched, when_not_matched
from snowflake.snowpark.types import IntegerType, StringType, StructType, StructField

def generate_attributes(session:snowflake.snowpark.Session, attr_table:str, target_collection_name:str, target_entity_name:str, version:str):
    try:

        # generates show columns statement for source table
        show_col_sql_text = f"show columns in table {attr_table}"

        session.sql(show_col_sql_text).collect()

        # use the last_query_id to get results of the show col statement
        last_query_results_sql_text = f"""
            select
                    '{target_collection_name}' as target_collection_name
                ,   '{target_entity_name}' as target_entity_name
                ,   "column_name" as target_entity_attribute_name
                ,   object_construct(
                        'data_type',parse_json("data_type"):type::varchar,
                        'is_nullable',parse_json("data_type"):nullable::boolean,
                        'precision',parse_json("data_type"):precision::number,
                        'scale',parse_json("data_type"):scale::number,
                        'length',parse_json("data_type"):length::number,
                        'byte_length',parse_json("data_type"):byteLength::number,
                        'description',null
                    ) as target_attribute_properties

            from table(RESULT_SCAN(LAST_QUERY_ID()))
        """

        source_df = session.sql(last_query_results_sql_text).with_column("LAST_UPDATED_TIMESTAMP", current_timestamp())

        target_df = session.table('data_model_mapper_admin_app.ADMIN.TARGET_ENTITY_ATTRIBUTE')
        target_df_collection = session.table('data_model_mapper_admin_app.ADMIN.TARGET_COLLECTION')
        target_df_entity = session.table('data_model_mapper_admin_app.ADMIN.TARGET_ENTITY')

        # merge with table on pk cols
        target_df.merge(
            source_df,
            (
                (target_df["TARGET_COLLECTION_NAME"] == source_df["TARGET_COLLECTION_NAME"]) &
                (target_df["TARGET_ENTITY_NAME"] == source_df["TARGET_ENTITY_NAME"]) &
                (target_df["TARGET_ENTITY_ATTRIBUTE_NAME"] == source_df["TARGET_ENTITY_ATTRIBUTE_NAME"])
            )
            ,
            [
                when_matched().update(
                    {
                        "TARGET_ATTRIBUTE_PROPERTIES": source_df["TARGET_ATTRIBUTE_PROPERTIES"],
                        "LAST_UPDATED_TIMESTAMP": source_df["LAST_UPDATED_TIMESTAMP"],
                        "VERSION": version           }
                ),
                when_not_matched().insert(
                    {
                        "TARGET_COLLECTION_NAME": source_df["TARGET_COLLECTION_NAME"],
                        "TARGET_ENTITY_NAME": source_df["TARGET_ENTITY_NAME"],
                        "TARGET_ENTITY_ATTRIBUTE_NAME": source_df["TARGET_ENTITY_ATTRIBUTE_NAME"],
                        "TARGET_ATTRIBUTE_PROPERTIES": source_df["TARGET_ATTRIBUTE_PROPERTIES"],
                        "VERSION": version
                    }
                )
            ]
        )

        source_df_text = f"""
            select
                    '{target_collection_name}' as target_collection_name
                ,   '{target_entity_name}' as target_entity_name
        """
        source_df_collect = session.sql(source_df_text).with_column("LAST_UPDATED_TIMESTAMP", current_timestamp())

        target_df_entity.merge(
            source_df_collect,
            (
                (target_df_entity["TARGET_COLLECTION_NAME"] == source_df_collect["TARGET_COLLECTION_NAME"]) &
                (target_df_entity["TARGET_ENTITY_NAME"] == source_df_collect["TARGET_ENTITY_NAME"])
            )
            ,
            [
                when_matched().update(
                    {
                        "LAST_UPDATED_TIMESTAMP": source_df_collect["LAST_UPDATED_TIMESTAMP"],
                        "VERSION": version             }
                ),
                when_not_matched().insert(
                    {
                        "TARGET_COLLECTION_NAME": source_df_collect["TARGET_COLLECTION_NAME"],
                        "TARGET_ENTITY_NAME": source_df_collect["TARGET_ENTITY_NAME"],
                        "VERSION": version
                    }
                )
            ]
        )

        target_df_collection.merge(
            source_df_collect,
            (
                (target_df_entity["TARGET_COLLECTION_NAME"] == source_df_collect["TARGET_COLLECTION_NAME"])
            )
            ,
            [
                when_matched().update(
                    {
                        "LAST_UPDATED_TIMESTAMP": source_df_collect["LAST_UPDATED_TIMESTAMP"],
                        "VERSION": version             }
                ),
                when_not_matched().insert(
                    {
                        "TARGET_COLLECTION_NAME": source_df_collect["TARGET_COLLECTION_NAME"],
                        "VERSION": version
                    }
                )
            ]
        )

        return "Operation Successful"
    except:
        return "Operation Failed"
$$
;

Let's add some helper objects for deploying/redeploying the streamlit app

In [ ]:
create or replace database data_model_mapper_admin_deployment_db comment='{"origin":"sf_sit","name":"dmm_admin","version":{"major":2, "minor":0},"attributes":{"component":"dmm"}}';
create or replace schema data_model_mapper_admin_deployment_db.code comment='{"origin":"sf_sit","name":"dmm_admin","version":{"major":2, "minor":0},"attributes":{"component":"dmm"}}';
drop schema if exists data_model_mapper_admin_deployment_db.public;
create or replace table data_model_mapper_admin_deployment_db.code.file (
        NAME varchar
    ,   CONTENT varchar(16777216)
) comment='{"origin":"sf_sit","name":"dmm","version":{"major":2, "minor":0},"attributes":{"component":"dmm_admin"}}';

/* constructs and puts files into a stage */
CREATE OR REPLACE PROCEDURE data_model_mapper_admin_deployment_db.code.PUT_TO_STAGE(STAGE VARCHAR,FILENAME VARCHAR, CONTENT VARCHAR)
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION=3.11
PACKAGES=('snowflake-snowpark-python')
HANDLER='put_to_stage'
COMMENT='{"origin":"sf_sit","name":"dmm","version":{"major":2, "minor":0},"attributes":{"component":"dmm_admin"}}'
AS $$
import io
import os

def put_to_stage(session, stage, filename, content):
    local_path = '/tmp'
    local_file = os.path.join(local_path, filename)
    f = open(local_file, "w", encoding='utf-8')
    f.write(content)
    f.close()
    session.file.put(local_file, '@'+stage, auto_compress=False, overwrite=True)
    return "saved file "+filename+" in stage "+stage
$$;

/* helper stored procedure to help create image files */
create or replace procedure data_model_mapper_admin_deployment_db.code.put_image_to_stage(stage varchar,filename varchar, base64_code varchar)
returns string
language python
runtime_version=3.8
packages=('snowflake-snowpark-python')
handler='put_image_to_stage'
COMMENT='{"origin":"sf_sit","name":"dmm","version":{"major":2, "minor":0},"attributes":{"component":"dmm_admin"}}'
AS $$
import io
import os
import base64


def put_image_to_stage(session, stage, filename, base64_code):
    local_path = '/tmp'
    local_file = os.path.join(local_path, filename)
    
    with open(local_file, "wb") as fw:
        img_data = base64_code.encode()
        fw.write(base64.b64decode(img_data))
        fw.close()
        session.file.put(local_file, '@'+stage, auto_compress=False, overwrite=True)
        return "saved file "+filename+" in stage "+stage
$$;

Now we will stage all of our app objects

In [ ]:
-- /* add environment file for streamlit - includes all references to libraries that the Streamlit needs */
insert into data_model_mapper_admin_deployment_db.code.file (name , content)
values ( 'ENVIRONMENT',$$
name: sf_env
channels:
- snowflake
dependencies:
- streamlit=1.42.0

 $$);

In [ ]:
/* add the Streamlit file */
insert into data_model_mapper_admin_deployment_db.code.file (name , content)
values ( 'STREAMLIT',$$
from appPages.page import st
from appPages.target_creation import TargetCreation
from appPages.target_administration import TargetAdministration
from appPages.target_entity_selection import TargetEntitySelection
from appPages.consumer_app_setup import ConsumerAppSetup
from snowflake.snowpark.context import get_active_session


def set_session():
    try:
        import snowflake.permissions as permissions

        session = get_active_session()

        st.session_state["streamlit_mode"] = "NativeApp"
    except:
        try:
            session = get_active_session()

            st.session_state["streamlit_mode"] = "SiS"
        except:
            import snowflake_conn as sfc

            session = sfc.init_snowpark_session("account_1")

            st.session_state["streamlit_mode"] = "OSS"

    return session


# Set starting page
if "page" not in st.session_state:
    st.session_state.page = "target_admin"

    #Set table database location session variables
    #This should happen on first load only
    st.session_state.native_database_name = "DATA_MODEL_MAPPER_ADMIN_APP"

pages = [
    TargetCreation(),
    TargetAdministration(),
    TargetEntitySelection(),
    ConsumerAppSetup()
]


def main():
    st.set_page_config(layout="wide")
    if 'session' not in st.session_state:
        st.session_state.session = set_session()
    for page in pages:
        if page.name == st.session_state.page:
            st.session_state.layout = "wide"
            page.print_sidebar()
            page.print_page()


main()

$$);

And add a stage for our version/patch with the files

In [ ]:
create or replace stage data_model_mapper_admin_deployment_db.code.files_v1_p0 comment='{"origin":"sf_sit","name":"dmm","version":{"major":2, "minor":0},"attributes":{"component":"dmm_admin"}}';

/* put files into stage */
CALL data_model_mapper_admin_deployment_db.code.PUT_TO_STAGE('files_v1_p0/code_artifacts/streamlit','environment.yml', (SELECT CONTENT FROM data_model_mapper_admin_deployment_db.code.file WHERE NAME = 'ENVIRONMENT'));
CALL data_model_mapper_admin_deployment_db.code.PUT_TO_STAGE('files_v1_p0/code_artifacts/streamlit','modeler_admin.py', (SELECT CONTENT FROM data_model_mapper_admin_deployment_db.code.file WHERE NAME = 'STREAMLIT'));

The code block below is if you want to use the file_pickler notebook to load your pickle files. If you want to use the load_db_file_picker notebook method comment out the file pickler notebook code block cell (25)

Stage pickle of Streamlit files and images for app.

Note, if you would like to confirm the pickle string is the same, please check out the file_pickler notebook and run it locally with the files. The resulting string should match.

In [ ]:
/* create the database and schema that will house our code for installation */
create or replace database data_model_mapper_admin_test_db comment='{"origin":"sf_sit","name":"dmm","version":{"major":2, "minor":0},"attributes":{"component":"dmm_admin"}}';
create or replace schema data_model_mapper_admin_test_db.code comment='{"origin":"sf_sit","name":"dmm","version":{"major":2, "minor":0},"attributes":{"component":"dmm_admin"}}';


create or replace table data_model_mapper_admin_test_db.code.file (
         NAME varchar
     ,   CONTENT varchar(16777216)
 ) comment='{"origin":"sf_sit","name":"dmm","version":{"major":2, "minor":0},"attributes":{"component":"dmm_admin"}}';

insert into data_model_mapper_admin_test_db.code.file (NAME, CONTENT)
values ('PICKLE', '...
')


In [ ]:
import pickle
import codecs
import os
import base64

from snowflake.snowpark.context import get_active_session
session = get_active_session()
session.sql('use role accountadmin;').collect()

def decode(obj):
    pickled = codecs.decode(obj.encode(), 'base64')
    return pickle.loads(pickled)

encoded_obj = session.sql("select content from data_model_mapper_admin_test_db.code.file where NAME = 'PICKLE'").collect()[0][0]

session.sql('use role accountadmin;').collect()

# Decode object to confirm it is the same as the original object
decoded_obj = decode(encoded_obj)

# Stage each file
for file_name, file_content in decoded_obj['files'].items():
    # replace double dollars to help with SQL compatibility
    file_content = str(file_content.decode("utf-8")).replace("$$", '"""')
    sql = """call data_model_mapper_admin_deployment_db.code.put_to_stage('files_v1_p0/code_artifacts/streamlit/appPages','{file_name}', $${file_content}$$)""".format(file_name=file_name, file_content=file_content)
    session.sql(sql).collect()

# Stage each image
for image_name, image_content in decoded_obj['images'].items():
    image_content = str(image_content.decode("utf-8"))
    sql = """call data_model_mapper_admin_deployment_db.code.put_image_to_stage('files_v1_p0/code_artifacts/streamlit/Images','{image_name}', $${image_content}$$)""".format(image_name=image_name, image_content=image_content)
    session.sql(sql).collect()

session.sql('use role accountadmin;').collect()
create_tbl_sql = """create or replace table data_model_mapper_admin_test_db.code.file (
        NAME varchar
    ,   CONTENT varchar(16777216)
) comment='{"origin":"sf_sit","name":"dmm","version":{"major":2, "minor":0},"attributes":{"component":"dmm_admin"}}';"""
session.sql(create_tbl_sql).collect()


Create a version and install the streamlit app

In [ ]:
use role accountadmin;
call system$wait(5);



/* create streamlit app */
CREATE OR REPLACE STREAMLIT data_model_mapper_admin
ROOT_LOCATION = '@data_model_mapper_admin_deployment_db.code.files_v1_p0/code_artifacts/streamlit' 
MAIN_FILE = '/modeler_admin.py'
QUERY_WAREHOUSE = APP_WH
COMMENT ='{"origin":"sf_sit","name":"dmm","version":{"major":2, "minor":0},"attributes":{"component":"dmm_admin"}}';